### air_data, weather_data, 발전량데이터 column 통일해서 merge 시키기

In [1]:
import os
import pandas as pd

In [7]:
# 경로 설정
META_DATA_PATH = '../data/solar_energy/meta_data.csv' 
GENERATION_DATA_PATH = '../data/solar_energy/preprocessed_data/' 
AIR_DATA_PATH = '../data/air_data/preprocessed_data/'
WEATHER_DATA_PATH = '../data/weather_data/preprocessed_data/'
OUTPUT_PATH = '../data/concat_data/'

meta_data = pd.read_csv(META_DATA_PATH)

In [8]:
# air, generation 데이터는 00:00가 24:00로 돼있어서 변환하고 merge해야함
def convert_24_to_00(df):
    """24:00 시간을 00:00으로 변환 (날짜는 그대로 유지)"""
    df = df.copy()
    
    # 24:00인 행 찾기
    mask_24 = df['time'] == '24:00'
    
    if mask_24.any():
        # 시간을 00:00으로 변경 (날짜는 그대로)
        df.loc[mask_24, 'time'] = '00:00'
    
    return df

In [9]:
for idx, row in meta_data.iterrows():
    plant_name = row['name']  # 발전소 이름 컬럼명이 'name'이라고 가정
    print(f'============={plant_name}=============')
    air_city = row['city']  
    air_district = row['district']
    air_location = air_city + ' ' + air_district
    weather_location = row['location_name']  
    weather_data = pd.read_parquet(os.path.join(WEATHER_DATA_PATH, f'{weather_location}2013_2025.parquet'))
    generation_data = pd.read_parquet(os.path.join(GENERATION_DATA_PATH, f'{plant_name}.parquet'))
    whole_air_data = pd.read_parquet(os.path.join(AIR_DATA_PATH, f'air_{air_city}.parquet'))
    air_data = whole_air_data[whole_air_data['지역'] == air_location]    
    air_data = air_data.drop_duplicates()  # 중복 제거
    if len(air_data)<1000:
        print('air 데이터 특정 지역꺼 잘 안받아와짐: ', plant_name,air_location)

    if len(air_data['측정소명'].unique())>1:
        # 측정소가 여러 개일 경우, 'PM25' 의 isna 개수가 제일 적은 측정소를 선택
        # 각 측정소별로 'PM25'의 NaN 개수 계산
        na_counts = air_data.groupby('측정소명')['PM25'].apply(lambda x: x.isna().sum())
        
        # NaN 개수가 가장 적은 측정소명 선택
        best_station = na_counts.idxmin()
        print(f'전체 측정소: {air_data["측정소명"].unique()}')
        print(f'선택된 측정소: {best_station}')
        # 해당 측정소 데이터만 필터링
        air_data = air_data[air_data['측정소명'] == best_station]        

    print(f'날씨 기간: {weather_data["date"].min()} ~ {weather_data["date"].max()}')
    print(f'발전량 기간: {generation_data["date"].min()} ~ {generation_data["date"].max()}')
    print(f'대기질 기간: {air_data["date"].min()} ~ {air_data["date"].max()}')

    # 시간 형식 통일: 24:00 -> 00:00
    generation_data = convert_24_to_00(generation_data)
    air_data = convert_24_to_00(air_data)
    weather_data = convert_24_to_00(weather_data)  # weather_data에도 24:00이 있을 수 있으므로
    
    # 날짜, 시간 순으로 정렬
    generation_data = generation_data.sort_values(['date', 'time']).reset_index(drop=True)
    air_data = air_data.sort_values(['date', 'time']).reset_index(drop=True)
    weather_data = weather_data.sort_values(['date', 'time']).reset_index(drop=True)    

    # 3개 merge
    core_weather = weather_data[['date','time','temperature','humidity','rn','ws','wd','pv','pa','ps','ss','icsr','dc10Tca','dc10LmcsCa','lcsCh','vs','ts','sunrise','sunset']]    
    core_air = air_data[['date','time','SO2','CO','O3','NO2',"PM10",'PM25','미세먼지','초미세먼지']]
    merge_ = pd.merge(generation_data,core_weather,on=['date','time'],how='inner')
    merge = pd.merge(merge_, core_air, on=['date', 'time'], how='inner')
    print(f'합쳐진 데이터의 기간: {merge["date"].min()} ~ {merge["date"].max()}')
    dup_count = merge.duplicated(subset=['date', 'time']).sum()
    print(f"중복된 (date, time) 조합 개수: {dup_count}")    
    print('\n')
    
    merge.to_parquet(os.path.join(OUTPUT_PATH, f'{plant_name}.parquet'), index=False)

=============감우리=============
전체 측정소: ['호암동' '칠금동' '중앙탑면' '살미면']
선택된 측정소: 중앙탑면
날씨 기간: 2013-01-01 ~ 2025-02-28
발전량 기간: 2021-08-05 ~ 2025-02-28
대기질 기간: 2020-02-01 ~ 2022-12-31
합쳐진 데이터의 기간: 2021-08-05 ~ 2022-12-31
중복된 (date, time) 조합 개수: 0


=============남제주소내=============
전체 측정소: ['이도동' '연동' '고산리' '노형로' '조천읍' '한림읍' '화북동' '애월읍']
선택된 측정소: 화북동
날씨 기간: 2013-01-01 ~ 2025-02-28
발전량 기간: 2013-01-01 ~ 2025-02-28
대기질 기간: 2022-02-01 ~ 2022-12-31
합쳐진 데이터의 기간: 2022-02-01 ~ 2022-12-31
중복된 (date, time) 조합 개수: 0


=============무릉리=============
전체 측정소: ['동홍동' '성산읍' '대정읍' '남원읍' '강정동']
선택된 측정소: 강정동
날씨 기간: 2013-01-01 ~ 2025-02-28
발전량 기간: 2021-05-01 ~ 2025-02-28
대기질 기간: 2022-02-01 ~ 2022-12-31
합쳐진 데이터의 기간: 2022-02-01 ~ 2022-12-31
중복된 (date, time) 조합 개수: 0


=============부산복합자재창고=============
날씨 기간: 2013-01-01 ~ 2025-02-28
발전량 기간: 2013-01-01 ~ 2025-02-28
대기질 기간: 2015-01-01 ~ 2022-12-31
합쳐진 데이터의 기간: 2015-01-01 ~ 2022-12-31
중복된 (date, time) 조합 개수: 0


=============부산수처리장=============
날씨 기간: 2013-01-01 ~ 2025-02-

In [5]:
sample = pd.read_parquet('../data/concat_data/무릉리.parquet')
sample.sort_values(by=['date','time'], inplace=True)
sample

,date,호기,총량(kw),평균(kw),최대(kw),최소(kw),최대(시간별_kw),최소(시간별_kw),value,time,...,sunrise,sunset,SO2,CO,O3,NO2,PM10,PM25,미세먼지,초미세먼지
0,2022-02-01,1,2593.967,108.082,430.279,0,430.279,2.169,0.0,00:00,...,07:30,18:05,0.001,0.4,0.046,0.006,35.0,NaN,보통,None
1,2022-02-01,1,2593.967,108.082,430.279,0,430.279,2.169,0.0,01:00,...,07:30,18:05,0.001,0.2,0.053,0.004,20.0,11.0,좋음,좋음
2,2022-02-01,1,2593.967,108.082,430.279,0,430.279,2.169,0.0,02:00,...,07:30,18:05,0.002,0.3,0.055,0.004,40.0,NaN,보통,None
3,2022-02-01,1,2593.967,108.082,430.279,0,430.279,2.169,0.0,03:00,...,07:30,18:05,0.002,0.5,0.056,0.005,67.0,55.0,보통,나쁨
4,2022-02-01,1,2593.967,108.082,430.279,0,430.279,2.169,0.0,04:00,...,07:30,18:05,0.001,0.5,0.056,0.005,70.0,56.0,보통,나쁨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8011,2022-12-31,1,3622.297,150.929,607.453,0,607.453,1.400,0.0,19:00,...,07:37,17:36,0.001,0.2,0.050,0.006,18.0,10.0,좋음,좋음
8012,2022-12-31,1,3622.297,150.929,607.453,0,607.453,1.400,0.0,20:00,...,07:37,17:36,0.001,0.2,0.045,0.006,22.0,11.0,좋음,좋음
8013,2022-12-31,1,3622.297,150.929,607.453,0,607.453,1.400,0.0,21:00,...,07:37,17:36,0.001,0.2,0.039,0.007,16.0,8.0,좋음,좋음
8014,2022-12-31,1,3622.297,150.929,607.453,0,607.453,1.400,0.0,22:00,...,07:37,17:36,0.001,0.2,0.039,0.007,16.0,6.0,좋음,좋음
